## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Touros,BR,-5.20,-35.46,77.00,83,75,10.29,broken clouds
1,1,Upernavik,GL,72.79,-56.15,44.08,71,85,10.58,overcast clouds
2,2,Leningradskiy,RU,69.38,178.42,49.05,76,4,4.00,clear sky
3,3,Eucaliptus,BO,-17.58,-67.52,55.33,36,0,8.39,clear sky
4,4,Ituacu,BR,-13.81,-41.30,74.59,58,43,9.95,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Touros,BR,-5.20,-35.46,77.00,83,75,10.29,broken clouds
7,7,San Quintin,MX,30.48,-115.95,79.29,66,1,9.48,clear sky
9,9,Hilo,US,19.73,-155.09,84.20,66,90,6.93,overcast clouds
18,18,Salalah,OM,17.02,54.09,77.00,100,90,4.70,light intensity drizzle
25,25,Semnan,IR,35.57,53.40,75.31,38,0,10.27,clear sky
30,30,Vaini,TO,-21.20,-175.20,77.00,88,40,10.29,scattered clouds
31,31,Jamestown,US,42.10,-79.24,82.00,51,20,8.05,few clouds
32,32,Lorengau,PG,-2.02,147.27,82.58,75,71,1.45,broken clouds
34,34,Albany,US,42.60,-73.97,84.99,44,0,6.71,clear sky
35,35,Kapaa,US,22.08,-159.32,86.00,62,20,13.87,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             246
City                   246
Country                246
Lat                    246
Lng                    246
Max Temp               246
Humidity               246
Cloudiness             246
Wind Speed             246
Current Description    246
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

## There are no empty rows

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Touros,BR,77.00,broken clouds,-5.20,-35.46,
7,San Quintin,MX,79.29,clear sky,30.48,-115.95,
9,Hilo,US,84.20,overcast clouds,19.73,-155.09,
18,Salalah,OM,77.00,light intensity drizzle,17.02,54.09,
25,Semnan,IR,75.31,clear sky,35.57,53.40,
30,Vaini,TO,77.00,scattered clouds,-21.20,-175.20,
31,Jamestown,US,82.00,few clouds,42.10,-79.24,
32,Lorengau,PG,82.58,broken clouds,-2.02,147.27,
34,Albany,US,84.99,clear sky,42.60,-73.97,
35,Kapaa,US,86.00,few clouds,22.08,-159.32,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   246
Country                246
Max Temp               246
Current Description    246
Lat                    246
Lng                    246
Hotel Name             246
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
new_column_order = ["City", "Country", "Max Temp", "Current Description", "Lat", "Lng", "Hotel Name"]
clean_hotel_df = hotel_df[new_column_order]
clean_hotel_df.head(10)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, -10), zoom_level=2.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))